<a href="https://colab.research.google.com/github/cha0stooo/f16flyingdatasets/blob/master/myADNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/cha0stooo/f16flyingdatasets.git


Cloning into 'f16flyingdatasets'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [5]:
import tensorflow as tf 
import numpy as np 
import scipy.io
import time
tf.random.set_seed(1)
np.random.seed(1)
LAM = 0.01
def getReLUDx(X):
    one = tf.ones_like(X)
    zero = tf.zeros_like(X)
    dX = tf.where(X>=0,x=one,y=zero)
    return dX

class LANet(object):
    #def __init__(self, *inputs, layers, input_dim):   
    def __init__(self, inputs, layers, ld_dim,lo_dim):       
        self.layers = layers       
        self.num_layers = len(self.layers)
        self.ld_dim = ld_dim
        self.lo_dim = lo_dim

        #sample_num = inputs.shape[0]

        X = inputs #np.concatenate(inputs, 1)
        self.X_mean = X.mean(0, keepdims=True)
        self.X_std = X.std(0, keepdims=True)

        self.weights = []
        self.biases = []
        self.bias_d = []

        for l in range(0,self.num_layers-1):
            in_dim = self.layers[l]
            out_dim = self.layers[l+1]
            #W = np.random.normal(size=[out_dim,in_dim])
            #He 初始化
            W =  np.random.randn(out_dim, in_dim) * np.sqrt(2 / in_dim)
            b = np.zeros([out_dim,1])
            #g = np.ones([1, out_dim])
            # tensorflow variables
            self.weights.append(tf.Variable(W, dtype=tf.float64, trainable=True))
            self.biases.append(tf.Variable(b, dtype=tf.float64, trainable=True))

        #定义一个ld的偏置
        bias_d = np.random.rand(1,3)
        self.bias_d.append(tf.Variable(bias_d, dtype=tf.float64, trainable=True))

    @tf.function     
    def __call__(self, *inputs):
        
        inputs = tf.concat(inputs, 1)
        H = inputs
        old_dhd0 = tf.eye(self.weights[0].shape[1], dtype=tf.float64)
        old_dhd0 = tf.expand_dims(old_dhd0,axis=0)
        old_dhd0 = tf.tile(old_dhd0,(H.shape[0],1,1))

        for l in range(0, self.num_layers-1):
            W = self.weights[l]
            b = self.biases[l]

            #1.将输入扩展为 样本数X维度X1
            H = tf.reshape(H,[H.shape[0],H.shape[1],1])
            #2.扩展权重矩阵为 样本数X输出维度X输入维度
            W = tf.expand_dims(W,axis=0)
            W = tf.tile(W,(H.shape[0],1,1))

            #3.扩展偏置向量 样本数X输出维度X1
            b = tf.expand_dims(b,axis=0)
            b = tf.tile(b,(H.shape[0],1,1))

            X = W@H+b       #f(X)  X=wh+b
            if l < self.num_layers-2: #若不是最后一层，则用relu激活函数
                H = tf.keras.activations.relu(X)   #NXdimX1
                dH = getReLUDx(X)                  #NXdimX1
                dH = tf.squeeze(dH,axis=2)     #NXdim     
                dH_diag = tf.linalg.diag(dH)
                dHdH0 = dH_diag@W   #tf.linalg.diag(dH)    NXdimXdim
                old_dhd0 = dHdH0@old_dhd0
                
            else:
                #lo层，线性激活函数
                X_lo = X[:,0:6,:]   #X_lo = X[:,0:self.lo_dim,:]
                H_lo = X_lo
                #dHdH0 = tf.eye(len(W))@W
                dHdH0_lo = W[:,0:6,:] #dHdH0_lo = W[:,0:self.lo_dim,:]
                old_dhd0_lo = dHdH0_lo@old_dhd0

                #Ld层，Relu激活函数
                # X_ld = X[:,self.lo_dim:self.lo_dim+3,:]
                # H_ld = tf.keras.activations.relu(X_ld)
                # dH_ld = getReLUDx(X_ld)                  #NXdimX1
                # dH_ld = tf.squeeze(dH_ld,axis=2)     #NXdim     
                # dH_diag_ld = tf.linalg.diag(dH_ld)
                # dHdH0_ld = dH_diag_ld@W[:,self.lo_dim:self.lo_dim+3,:]   #tf.linalg.diag(dH)    NXdimXdim
                # old_dhd0_ld = dHdH0_ld@old_dhd0
                
        Y_lo = tf.squeeze(H_lo,axis=2)   #NXdim
        dY_lo = old_dhd0_lo

        # Y_ld = tf.squeeze(H_ld,axis=2)+self.bias_d[0]   #NXdim
        # dY_ld = old_dhd0_ld
        lo = Y_lo[:,0:self.lo_dim]
        ld = Y_lo[:,self.lo_dim:self.lo_dim+3]+self.bias_d[0]

        d_lo = dY_lo[:,0:self.lo_dim,:]
        d_ld = dY_lo[:,self.lo_dim:self.lo_dim+3,:]
        return lo,d_lo,ld,d_ld

    def save_model(self,path=''):
        #保存模型参数
        W = self.weights
        b = self.biases
        bd = self.bias_d
   
        W_dict = {}
        for i in range(len(W)):
            W_dict['w'+str(i)] = W[i].numpy()

        b_dict = {}
        for i in range(len(W)):
            b_dict['b'+str(i)] = b[i].numpy()
        
        bd_dict = {}
        bd_dict['bd'] = bd[0].numpy()

        total_dict = {}
        total_dict.update(W_dict)
        total_dict.update(b_dict)
        total_dict.update(bd_dict)

        scipy.io.savemat('model_paras.mat' , total_dict)

        from google.colab import files
        files.download('model_paras.mat')

    @tf.function
    def get_l2_loss(self):
        W = self.weights
        l2_loss = 0
        for _,wi in enumerate(W):
            l2_loss = l2_loss+tf.norm(wi)**2
        return l2_loss

class vehicle_net():
    def __init__(self,inputs,layers,ld_dim,lo_dim,hp):
        
        self.epochs = hp["epochs"]
        self.tf_optimizer = tf.keras.optimizers.Adam(
            learning_rate=hp["tf_lr"],
            beta_1=hp["tf_b1"],
            epsilon=hp["tf_eps"])
        self.LAM = hp["lamda"]
        #self.batchsize = hp["batchsize"]

        self.model = LANet(inputs,layers,ld_dim,lo_dim)
    @tf.function
    def cal_inverse_dynamic(self,q,dq,ddq):
        q = tf.convert_to_tensor(q)
        dq = tf.convert_to_tensor(dq)   
        dq = tf.reshape(dq,[dq.shape[0],dq.shape[1],1])  #NX3 -> NX3X1
        ddq = tf.convert_to_tensor(ddq)
        ddq = tf.reshape(ddq,[ddq.shape[0],ddq.shape[1],1])  #NX3 -> NX3X1

        Y_lo,dY_lo,Y_ld,dY_ld = self.model(q) 
        #计算逆动力学方程中的各项：M = J*ddq-0.5*[dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq ...]+dJ*dq
        #1.J = L*L'
        L_mat,L_mat_T,J = self.cal_J(Y_lo,Y_ld)

        #2.dJ/dt = dLdt@L'+L@dL'dt = dLdt@L'+L@dLdt'
        dl11 = tf.squeeze(dY_ld[:,0:1,:]@dq,axis=2)   #NX1X1 -> NX1
        dl22 = tf.squeeze(dY_ld[:,1:2,:]@dq,axis=2)
        dl33 = tf.squeeze(dY_ld[:,2:3,:]@dq,axis=2)

        dl21 = tf.squeeze(dY_lo[:,0:1,:]@dq,axis=2)
        dl31 = tf.squeeze(dY_lo[:,1:2,:]@dq,axis=2)
        dl32 = tf.squeeze(dY_lo[:,2:3,:]@dq,axis=2)

        dl12 = tf.zeros((dl11.shape[0],1),dtype = tf.float64)
        dl13 = tf.zeros((dl11.shape[0],1),dtype = tf.float64)
        dl23 = tf.zeros((dl11.shape[0],1),dtype = tf.float64)

        dl = tf.stack([dl11,dl12,dl13,dl21,dl22,dl23,dl31,dl32,dl33],axis=1)
        dl = tf.squeeze(dl,axis=2)
        dl_mat = tf.reshape(dl,[dl.shape[0],3,3])

        dl_mat_T = tf.transpose(dl_mat,perm=[0,2,1])  #将NX3X3 中第二维和第三维进行转置 

        dJ = dl_mat@L_mat_T+L_mat@dl_mat_T

        #3.(dq'@J@dq)/dq
        dLo_dq1 = dY_lo[:,:,0]     #NX3
        dLd_dq1 = dY_ld[:,:,0]
        dl_dq1_mat = self.vec2mat(dLd_dq1,dLo_dq1)   #dl_dq1

        dLo_dq2 = dY_lo[:,:,1]
        dLd_dq2 = dY_ld[:,:,1]
        dl_dq2_mat = self.vec2mat(dLd_dq2,dLo_dq2)

        dLo_dq3 = dY_lo[:,:,2]
        dLd_dq3 = dY_ld[:,:,2]
        dl_dq3_mat = self.vec2mat(dLd_dq3,dLo_dq3)

        #计算dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq
        dq_T = tf.transpose(dq,perm=[0,2,1])    #NX3X1 ->NX1X3
        tmp1 = dq_T@(dl_dq1_mat@L_mat_T+L_mat@tf.transpose(dl_dq1_mat,perm=[0,2,1]))@dq   #NX1X1
        tmp2 = dq_T@(dl_dq2_mat@L_mat_T+L_mat@tf.transpose(dl_dq2_mat,perm=[0,2,1]))@dq   #NX1X1
        tmp3 = dq_T@(dl_dq3_mat@L_mat_T+L_mat@tf.transpose(dl_dq3_mat,perm=[0,2,1]))@dq   #NX1X1
        tmp = tf.stack([tmp1,tmp2,tmp3],axis=1)
        tmp = tf.squeeze(tmp,axis=3)   #NX3X1X1  -> NX3X1
        #-0.5*[dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq ...]   #NX3X1
        f2 = -0.5*tmp

        #4.计算M = J*ddq-0.5*[dq_T*(dl_dq1*L_T+L*dL_dq1_T)*dq ...]+dJ*dq
        M_pre = J@ddq+f2+dJ@dq

        gama,phi = q[:,0:1],q[:,1:2]
        r11 = tf.ones_like(gama)
        r12 = tf.zeros_like(gama)
        r13 = -tf.sin(phi)
        r21 = tf.zeros_like(gama)
        r22 = tf.cos(gama)
        r23 = tf.sin(gama)*tf.cos(phi)
        r31 = tf.zeros_like(gama)
        r32 = -tf.sin(gama)
        r33 = tf.cos(gama)*tf.cos(phi)

        R = tf.stack([r11,r12,r13,r21,r22,r23,r31,r32,r33],axis=1)
        R = tf.squeeze(R,axis=2)
        R = tf.reshape(R,[R.shape[0],3,3])
        R_T = tf.transpose(R,perm=[0,2,1])  #将NX3X3 中第二维和第三维进行转置

        M_pre = tf.linalg.inv(R_T)@M_pre

        return M_pre    #NX3X1

    @tf.function
    def vec2mat(self,dLd_dq1,dLo_dq1):
        #将N个3X1的dld_dqi 以及dlo_dqi 组合成 N个 3X3的矩阵
        dl_dq11 = dLd_dq1[:,0:1]
        dl_dq12 = tf.zeros((dl_dq11.shape[0],1),dtype = tf.float64)
        dl_dq13 = tf.zeros((dl_dq11.shape[0],1),dtype = tf.float64)

        dl_dq21 = dLo_dq1[:,0:1]
        dl_dq22 = dLd_dq1[:,1:2]
        dl_dq23 = tf.zeros((dl_dq11.shape[0],1),dtype = tf.float64)

        dl_dq31 = dLo_dq1[:,1:2]
        dl_dq32 = dLo_dq1[:,2:3]
        dl_dq33 = dLd_dq1[:,2:3]

        dl = tf.stack([dl_dq11,dl_dq12,dl_dq13,dl_dq21,dl_dq22,dl_dq23,dl_dq31,dl_dq32,dl_dq33],axis=1)
        dl = tf.squeeze(dl,axis=2)
        dl_dq_mat = tf.reshape(dl,[dl.shape[0],3,3])

        return dl_dq_mat

    @tf.function  
    def cal_J(self,Lo,Ld):
       
        l1 = Ld[:,0:1]
        l2 = tf.zeros((l1.shape[0],1),dtype = tf.float64)
        l3 = tf.zeros((l1.shape[0],1),dtype = tf.float64)
        l4 = Lo[:,0:1]
        l5 = Ld[:,1:2]
        l6 = tf.zeros((l1.shape[0],1),dtype = tf.float64)
        l7 = Lo[:,1:2]
        l8 = Lo[:,2:3]
        l9 = Ld[:,2:3]

        L_vec = tf.stack([l1,l2,l3,l4,l5,l6,l7,l8,l9],axis=1)
        L_vec = tf.squeeze(L_vec,axis=2)
        L_mat = tf.reshape(L_vec,[L_vec.shape[0],3,3])

        L_vec_T = tf.stack([l1,l4,l7,l2,l5,l8,l3,l6,l9],axis=1)
        L_vec_T = tf.squeeze(L_vec_T,axis=2)
        L_mat_T = tf.reshape(L_vec_T,[L_vec_T.shape[0],3,3])

        J_mat = L_mat@L_mat_T

        #J_vec_total = tf.reshape(J_mat,[J_mat.shape[0],9])
        return L_mat,L_mat_T,J_mat  

    @tf.function
    def train(self,q,dq,ddq,mom0):

        with tf.GradientTape() as tp:
            M_pre0 = self.cal_inverse_dynamic(q,dq,ddq)
            #M_pre = tf.squeeze(M_pre0,axis=2)

            #四种计loss方式等价
            #loss = (tf.keras.losses.MSE(M_pre[:,0],mom[:,0])\
                # +tf.keras.losses.MSE(M_pre[:,1],mom[:,1])\
                # +tf.keras.losses.MSE(M_pre[:,2],mom[:,2]))/3

            #loss = tf.reduce_mean(tf.keras.losses.MSE(mom,M_pre))
            loss = tf.reduce_mean(tf.keras.losses.MSE(mom0,M_pre0))+LAM*self.model.get_l2_loss()
            # loss0 = tf.norm(M_pre[:,0]-mom[:,0])**2/samp_num + \
            #     tf.norm(M_pre[:,1]-mom[:,1])**2/samp_num+\
            #         tf.norm(M_pre[:,2]-mom[:,2])**2/samp_num

        variables = self.model.weights+self.model.biases+self.model.bias_d
        grads = tp.gradient(loss,variables)
        self.tf_optimizer.apply_gradients(grads_and_vars=zip(grads,variables))
        del tp
        #print(epoch,loss.numpy())
        return loss



    def fit(self,q,dq,ddq,Mom):     
        loss_save = []
        N_data = q.shape[0]
        for ep in range(self.epochs):
            #idx_data = np.random.choice(N_data, min(mini_batch_size, N_data))
            qi   = q#[idx_data,:]
            dqi  = dq#[idx_data,:]
            ddqi = ddq#[idx_data,:]
            Momi = Mom#[idx_data,:]
            Momi = tf.reshape(Momi,[Momi.shape[0],Momi.shape[1],1])
            time1 = time.time()                           
            loss = self.train(qi,dqi,ddqi,Momi)                
            loss_numpy = loss.numpy()
            loss_save.append(loss_numpy) 
            time2 = time.time()
            tt1 = time2-time1
            if ep%100 == 0:
                tf.print(tt1,ep,loss_numpy)
        return loss_save

def main():
    #1.数据准备
    state_roll = np.loadtxt('./f16flyingdatasets/state_roll2020-9-16.dat')
    state_pitch = np.loadtxt('./f16flyingdatasets/state_pitch2020-9-16.dat')
    state_yaw = np.loadtxt('./f16flyingdatasets/state_yaw2020-9-16.dat')
    state = np.vstack((state_roll,state_pitch,state_yaw))

    state = state_roll

    q = state[:,0:3]
    dq = state[:,3:6]
    ddq = state[:,6:9]
    mom = state[:,9:12]
    #mom0 = tf.reshape(mom,[mom.shape[0],mom.shape[1],1])
    #2.网络初始化
    layers = [3]+9*[9]+[9]
    hp = {}
    hp["epochs"] = 50000
    hp["tf_lr"] = 0.01
    hp["tf_b1"] = 0.9
    hp["tf_eps"] = 0.000000001
    hp["lamda"] = 0.01
    #hp["batchsize"] = q.shape[0]
    vnet = vehicle_net(q,layers,3,3,hp)
    
    #3.训练
    losses = vnet.fit(q,dq,ddq,mom)

    #保存结果
    #np.savetxt('loss.dat',losses)
    path = ''
    vnet.model.save_model(path)


main()
    

8.668675422668457 0 1517414984.693989
0.008353948593139648 100 1459860.938846928
0.008254051208496094 200 1033323.7602475117
0.011938095092773438 300 887754.4109884168
0.008496522903442383 400 796918.957335403
0.008621454238891602 500 732773.2915850626
0.008372783660888672 600 686798.5152995144
0.008487224578857422 700 658836.0403809
0.008381843566894531 800 629593.5664172362
0.00848531723022461 900 604707.6997797583
0.008999347686767578 1000 580250.941600225
0.008699893951416016 1100 552324.0020076266
0.008691549301147461 1200 513259.9914763403
0.008586645126342773 1300 422061.47062677477
0.008472681045532227 1400 131313.02079494574
0.010470151901245117 1500 142499.06554634892
0.008379936218261719 1600 42176.77639942323
0.008635282516479492 1700 41275.25563878283
0.008513689041137695 1800 96849.28749070039
0.00843667984008789 1900 225577.07109041113
0.008435964584350586 2000 29260.054687026513
0.008397817611694336 2100 38794.58495152651
0.008414030075073242 2200 76952.90229311003
0.00

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>